In [1]:
import numpy as np
import pandas as pd

In [2]:
df= pd.read_csv("D:\\Jupyter\\Test\\userbehaviour.csv")
df.head()

,userid,Average Screen Time,Average Spent on App (INR),Left Review,Ratings,New Password Request,Last Visited Minutes,Status
0,1001,17.0,634.0,1,9,7,2990,Installed
1,1002,0.0,54.0,0,4,8,24008,Uninstalled
2,1003,37.0,207.0,0,8,5,971,Installed
3,1004,32.0,445.0,1,6,2,799,Installed
4,1005,45.0,427.0,1,5,6,3668,Installed


In [3]:
df.isnull().sum()

userid                        0
Average Screen Time           0
Average Spent on App (INR)    0
Left Review                   0
Ratings                       0
New Password Request          0
Last Visited Minutes          0
Status                        0
dtype: int64

In [4]:
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"

# Let's Start by looking ar the highest, lowest and average screen time og all the users.

In [5]:
print(f"Average Screen Time = {df["Average Screen Time"].mean()}")
print(f"Average Screen Time = {df["Average Screen Time"].max()}")
print(f"Average Screen Time = {df["Average Screen Time"].min()}")

Average Screen Time = 24.39039039039039
Average Screen Time = 50.0
Average Screen Time = 0.0


# Now let's have a look at the highest, lowest and the average amount apent by all the users.

In [6]:
print(f'Average Spend of the Users = {df["Average Spent on App (INR)"].mean()}')
print(f'Highest Spend of the Users = {df["Average Spent on App (INR)"].max()}')
print(f'Lowest Spend of the Users = {df["Average Spent on App (INR)"].min()}')

Average Spend of the Users = 424.4154154154154
Highest Spend of the Users = 998.0
Lowest Spend of the Users = 0.0


# Now let's have a look at the relationship between the spending capacity and screen time of the active users and the users who have uninstalled the app.

In [7]:
figure = px.scatter(data_frame=df, x="Average Screen Time",
                      y="Average Spent on App (INR)",
                      size = "Average Spent on App (INR)",
                      color="Status",
                      title="Relationship between Spending Capacity and Screen Time",
                      trendline="ols")

figure.show()

# Now let's have a look at the relationship between the ratings given by users and the average screen time.

In [8]:
figure = px.scatter(data_frame=df, x="Average Screen Time",
                    y="Ratings",
                    size="Ratings",
                    color="Status",
                    title="Relationship between Ratings and Screen Time",
                    trendline="ols")

figure.show()

# App User Segmentation to find Retained and Lost Users

In [9]:
clustering_data = df[["Average Screen Time", "Left Review",
                      "Ratings", "Last Visited Minutes",
                      "Average Spent on App (INR)",
                      "New Password Request"]]

In [10]:
from sklearn.preprocessing import MinMaxScaler

for i in clustering_data.columns:
    MinMaxScaler(i)

from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(clustering_data)
df["Segments"] = clusters

print(df.head(10))

   userid  Average Screen Time  Average Spent on App (INR)  Left Review  \
0    1001                 17.0                       634.0            1   
1    1002                  0.0                        54.0            0   
2    1003                 37.0                       207.0            0   
3    1004                 32.0                       445.0            1   
4    1005                 45.0                       427.0            1   
5    1006                 28.0                       599.0            0   
6    1007                 49.0                       887.0            1   
7    1008                  8.0                        31.0            0   
8    1009                 28.0                       741.0            1   
9    1010                 28.0                       524.0            1   

   Ratings  New Password Request  Last Visited Minutes       Status  Segments  
0        9                     7                  2990    Installed         0  
1        4    

In [11]:
print(df["Segments"].value_counts())

Segments
0    910
2     45
1     44
Name: count, dtype: int64


# Noe let's rename the segments for a better understanding

In [12]:
df["Segments"] = df["Segments"].map({0: "Retained",
                        1: "Churn",
                        2: "Needs Attention"})

In [17]:
PLOT = go.Figure()

for i in list(df["Segments"].unique()):
    PLOT.add_trace(go.Scatter(x=df[df["Segments"] == i]["Last Visited Minutes"],
                                y=df[df["Segments"] == i]["Average Spent on App (INR)"],
                                mode="markers",marker_size=6,marker_line_width=1,
                                name=str(i)))
    
PLOT.update_traces(hovertemplate='Last Visited Minutes: %{x}<br>Average Spent on App (INR): %{y}')

PLOT.update_layout(width=800, height=800,autosize=True,showlegend=True,
                   yaxis_title="Average Spent on App (INR)",
                     xaxis_title="Last Visited Minutes",
                     scene=dict(xaxis=dict(title="Last Visited Minutes",title_font_color="black"),
                                    yaxis=dict(title="Average Spent on App (INR)",title_font_color="black")))